In [1]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: E:\vscode\workspace\main-newest\mlfs-book
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-18 15:17:03,296 INFO: Initializing external client
2025-11-18 15:17:03,297 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-18 15:17:04,947 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286333


'{"country": "China", "city": "Shanghai", "street": "Jiading Nanxiang", "aqicn_url": "https://api.waqi.info/feed/@10208", "latitude": 31.23, "longitude": 121.47}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

In [5]:
air_quality_3mean_and_weather_fg = fs.get_feature_group(
    name='air_quality_3mean_and_weather',
    version=1,
)

---

In [6]:
from datetime import timedelta, datetime

# df_max_date = air_quality_3mean_and_weather_fg.select(["date"]).read()
# df_max_date["date"] = pd.to_datetime(df_max_date["date"])
# last_processed_date = df_max_date["date"].max()
# print(f"The latest date in combined features: {last_processed_date.date()}")
# context_start_date = last_processed_date - timedelta(days=5)
start_date = pd.to_datetime("today") - pd.Timedelta(days=7)
df_history_aq = air_quality_fg.select(["date", "city", "pm25"]).filter(air_quality_fg.date >= start_date).read()
df_history_aq["date"] = pd.to_datetime(df_history_aq["date"])

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.89s) 


In [7]:
# query_aq = air_quality_fg.select(["date", "pm25"]).filter(air_quality_fg.date >= context_start_date)
# df_aq = query_aq.read()

# query_weather = weather_fg.select_all().filter(weather_fg.date >= context_start_date)
# df_weather = query_weather.read()

# df_aq["date"] = pd.to_datetime(df_aq["date"])
# df_weather["date"] = pd.to_datetime(df_weather["date"])

# df_aq_processed = df_aq.set_index("date").sort_index()
# df_aq_processed['pm25'] = df_aq_processed['pm25'].ffill()
# df_aq_processed = df_aq_processed.fillna(method='ffill')
# df_aq_processed['pm25_lag_3_mean'] = df_aq_processed['pm25'].rolling(
#     window=3, 
#     min_periods=1 
# ).mean().shift(1)
# df_aq_processed = df_aq_processed.reset_index().rename(columns={'index': 'date'})
# df_aq_processed

In [8]:
# df_final = pd.merge(
#     df_aq_processed, 
#     df_weather, 
#     on=["date", "city"],
#     how="inner"
# )

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [9]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,72.0,China,Shanghai,Jiading Nanxiang,2025-11-18,https://api.waqi.info/feed/@10208


In [10]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [11]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 31.25°N 121.5°E
Elevation 3.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-18,6.05,0.0,10.829958,291.447815,Shanghai
1,2025-11-19,7.05,0.0,7.145796,319.085693,Shanghai
2,2025-11-20,8.15,0.0,1.297998,213.690094,Shanghai
3,2025-11-21,10.70,0.0,6.849467,356.987274,Shanghai
4,2025-11-22,11.40,0.0,6.369050,132.709366,Shanghai
5,2025-11-23,15.55,0.0,3.758510,286.699310,Shanghai
6,2025-11-24,10.75,0.0,15.021105,314.029053,Shanghai


In [12]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 352.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [13]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-18 15:17:13,140 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286333/fs/1273958/fg/1721783


Uploading Dataframe: 100.00% |████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286333/jobs/named/air_quality_1_offline_fg_materialization/executions


(Job('air_quality_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 761942
         }
       },
       "result": {
         "observed_value": 72.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T02:17:13.000139Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectat

In [14]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-18 15:17:26,919 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286333/fs/1273958/fg/1721784


Uploading Dataframe: 100.00% |████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286333/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-18 15:17:44,287 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-18 15:17:47,475 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 15:19:39,723 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-18 15:19:39,913 INFO: Waiting for log aggregation to finish.
2025-11-18 15:20:01,987 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 761943
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T02:17:26.000919Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store


In [15]:
df_history_aq['date'] = pd.to_datetime(df_history_aq['date'])
aq_today_df['date'] = pd.to_datetime(aq_today_df['date'])

In [16]:
target_date = aq_today_df['date'].iloc[0]

In [17]:
if target_date in df_history_aq['date'].values:
    print(f"skip concatenating")
    df_combined = df_history_aq.copy()
else:
    print(f"concatenating")
    df_combined = pd.concat([df_history_aq, aq_today_df], ignore_index=True)

skip concatenating


In [18]:

df_combined["date"] = pd.to_datetime(df_combined["date"])
# df_combined = df_combined.sort_values("date").drop_duplicates(["date", "city"])
df_calc = df_combined.set_index("date").sort_index()

df_calc = df_calc.asfreq('D')
df_calc['pm25'] = df_calc['pm25'].ffill()

if 'city' in df_calc.columns:
    df_calc['city'] = df_calc['city'].ffill()

df_calc['pm25_lag_3_mean'] = df_calc['pm25'].rolling(window=3, min_periods=1).mean().shift(1)

today_date = pd.to_datetime(aq_today_df['date'].iloc[0]).tz_localize(None)
df_calc = df_calc.reset_index()
df_calc['date'] = pd.to_datetime(df_calc['date']).dt.tz_localize(None)
df_today_features = df_calc[df_calc['date'] == today_date].copy()
# if 'city' in df_today_features.columns:
#     df_today_features['city'] = df_today_features['city'].ffill()

df_features_only = df_today_features[['date', 'city', 'pm25_lag_3_mean']]

df_final_upload = pd.merge(
    df_features_only,      
    daily_df,        
    on=["date", "city"],
    how="inner"
    )

df_final_upload

,date,city,pm25_lag_3_mean,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant
0,2025-11-18,Shanghai,54.0,6.05,0.0,10.829958,291.447815


In [19]:
air_quality_3mean_and_weather_fg.insert(df_final_upload, wait=True)

2025-11-18 15:20:02,295 INFO: 	3 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286333/fs/1273958/fg/1737065


Uploading Dataframe: 100.00% |████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_3mean_and_weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286333/jobs/named/air_quality_3mean_and_weather_1_offline_fg_materialization/executions
2025-11-18 15:20:20,290 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-18 15:20:26,664 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 15:22:15,105 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2025-11-18 15:22:18,283 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-18 15:22:18,450 INFO: Waiting for log aggregation to finish.
2025-11-18 15:22:37,072 INFO: Execution finished successfully.


(Job('air_quality_3mean_and_weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25_lag_3_mean",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 768025
         }
       },
       "result": {
         "observed_value": 54.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T02:20:02.000294Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_ty